In [1]:
import pandas as pd
df = pd.read_csv("/Users/ellise/Desktop/untitled folder 2/df00.csv")
df.drop("Unnamed: 0",axis = 1).head(5)

,VernacularNameCategory,latitude,longitude,DepthInMeters,Oxygen,Salinity,Temperature,Phosphate,LocationAccuracy,Station_-,...,SamplingEquipment_submersible,SamplingEquipment_towed camera,SamplingEquipment_trawl,RecordType_NaN,RecordType_catch record,RecordType_literature,RecordType_notation,RecordType_specimen,RecordType_still image,RecordType_video observation
0,2,36.87485,-121.92476,48.0,5.605830,33.207511,11.375480,0.874281,100.0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,25.88676,-167.78026,2000.0,1.469286,34.564939,2.561229,3.166553,50.0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,37.36785,-123.39338,1294.0,0.896997,34.496119,3.085411,3.205367,100.0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,51.50220,-177.92657,1078.0,0.506452,34.402537,3.266718,3.181331,20.0,0,...,0,0,0,0,0,0,0,0,0,1
4,7,29.06941,-88.37699,404.0,0.902513,34.207603,7.068033,2.990354,50.0,0,...,0,0,0,1,0,0,0,0,0,0


In [2]:
import numpy as np
X = np.array(df.drop(['VernacularNameCategory'],axis=1))
Y = np.array(df["VernacularNameCategory"])
Y = Y.astype(int)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', Y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', Y_test.shape)

('Train data shape: ', (6183, 1191))
('Train labels shape: ', (6183,))
('Test data shape: ', (1092, 1191))
('Test labels shape: ', (1092,))


In [3]:
import torch
from torch.autograd import Variable

a = torch.from_numpy(X_test)
b = torch.from_numpy(Y_test)

In [4]:
dtype = torch.FloatTensor
a = a.type(dtype)

In [34]:
import random
import torch.nn as nn
from torch.autograd import Variable

all_losses = []

In [50]:
model(x)

Variable containing:
-2.3244e+01 -2.2885e+01 -3.9571e+01  ...  -4.2763e+01 -4.1087e+01 -1.4885e+01
-1.5521e+01 -1.5266e+01 -2.6386e+01  ...  -2.8545e+01 -2.7408e+01 -9.9174e+00
-7.3062e+00 -7.1641e+00 -1.2365e+01  ...  -1.3410e+01 -1.2842e+01 -4.6197e+00
                ...                   ⋱                   ...                
-1.3281e+01 -1.3421e+01 -2.2693e+01  ...  -2.4627e+01 -2.3763e+01 -8.4799e+00
-2.9935e+00 -2.9689e+00 -4.8045e+00  ...  -5.2563e+00 -5.0302e+00 -1.9564e+00
-7.3066e+00 -7.2854e+00 -1.2359e+01  ...  -1.3498e+01 -1.2986e+01 -4.6697e+00
[torch.FloatTensor of size 1092x10]

In [35]:
# -*- coding: utf-8 -*-
import random
import torch.nn as nn
from torch.autograd import Variable


class DynamicNet(torch.nn.Module):
    def __init__(self, D_in, H, D_out):
        """
        In the constructor we construct three nn.Linear instances that we will use
        in the forward pass.
        """
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
        and reuse the middle_linear Module that many times to compute hidden layer
        representations.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same Module many
        times when defining a computational graph. This is a big improvement from Lua
        Torch, where each Module could be used only once.
        """
        h_relu = self.input_linear(x).clamp(min=0)#Clamps all elements into the range [min_value, max_value](ReLU here)
        for _ in range(random.randint(0, 5)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        y_pred = self.softmax(y_pred)
        return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 6183, 1191, 400, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(a)
y = Variable(b, requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-10, momentum=0.9)
for t in range(100):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])
    all_losses.append(loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


(0, 1132.772705078125)
(1, 1132.39111328125)
(2, 32.291748046875)
(3, 1131.0184326171875)
(4, 120.41736602783203)
(5, 241.85568237304688)
(6, 241.7765350341797)
(7, 2272.981689453125)
(8, 2271.31884765625)
(9, 32.27882766723633)
(10, 2265.51171875)
(11, 284.77850341796875)
(12, 241.1358184814453)
(13, 32.271976470947266)
(14, 284.69732666015625)
(15, 240.73695373535156)
(16, 120.32613372802734)
(17, 1122.2784423828125)
(18, 32.250492095947266)
(19, 32.2459831237793)
(20, 120.28337860107422)
(21, 120.27188110351562)
(22, 1119.438720703125)
(23, 284.4140319824219)
(24, 120.23226928710938)
(25, 1117.2906494140625)
(26, 32.21677780151367)
(27, 284.2856140136719)
(28, 284.2456970214844)
(29, 284.1948547363281)
(30, 2228.987060546875)
(31, 239.0393524169922)
(32, 120.13055419921875)
(33, 283.9734191894531)
(34, 238.6962127685547)
(35, 120.0915756225586)
(36, 2219.00244140625)
(37, 1109.9468994140625)
(38, 1109.2005615234375)
(39, 283.6619873046875)
(40, 32.15470886230469)
(41, 283.5602111816

In [ ]:
topk_n, topk_i = model(x).data.topk(1)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(all_losses)

In [ ]:
count = 0
for i in xrange(Y.size):
    if topk_i[i][0] == Y[i]:
        count+=1

print(count)

In [ ]:
topk_i[3][0] != Y[3]

In [ ]:
torch.cuda.is_available()

In [ ]:
from torch.autograd import Variable
x = Variable(torch.ones(2, 2), requires_grad = True)
y = x + 2
y.grad_fn

# y 是作为一个操作的结果创建的因此y有一个creator 
z = y * y * 3
out = z.mean()

# 现在我们来使用反向传播
out.backward()

# out.backward()和操作out.backward(torch.Tensor([1.0]))是等价的
# 在此处输出 d(out)/dx
x.grad
#只有x是variable所以有grad方法

In [ ]:
x = torch.randn(3)
x = Variable(x, requires_grad = True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)
x.grad

In [57]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1191, 1191, 1) # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv1d(1191, 1191, 1)
        self.fc1   = nn.Linear(1191, 800) # an affine operation: y = Wx + b
        self.fc2   = nn.Linear(800, 400)
        self.fc3   = nn.Linear(400, 10)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = F.relu(self.conv1(x)) # Max pooling over a (2, 2) window z 特征减一半
        x = F.relu(self.conv2(x)) # If the size is a square you can only specify a single number
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

x = Variable(c)
y = Variable(b, requires_grad=False)

# Construct our model by instantiating the class defined above
model = Net()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)
for t in range(100):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    print(t, loss.data[0])
    all_losses.append(loss.data[0])

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

(0, 305.6327819824219)
(1, 296.2607116699219)
(2, 570.933837890625)
(3, 882.069091796875)
(4, 886.6889038085938)
(5, 1013.9261474609375)
(6, 848.6907958984375)
(7, 655.14013671875)
(8, 449.9483642578125)
(9, 125.31603240966797)
(10, 264.0281677246094)
(11, 357.7687683105469)
(12, 155.2150115966797)
(13, 159.3505859375)
(14, 138.4070281982422)
(15, 283.1252136230469)
(16, 241.81387329101562)
(17, 189.71304321289062)
(18, 220.26724243164062)
(19, 182.927734375)
(20, 176.0771942138672)
(21, 120.96100616455078)
(22, 83.7100830078125)
(23, 82.14942932128906)
(24, 45.70379638671875)
(25, 40.27718734741211)
(26, 49.20444107055664)
(27, 58.009647369384766)
(28, 32.03042984008789)
(29, 36.469242095947266)
(30, 41.17305374145508)
(31, 36.19930648803711)
(32, 38.100650787353516)
(33, 32.48198318481445)
(34, 33.0024528503418)
(35, 19.986068725585938)
(36, 20.881635665893555)
(37, 12.609193801879883)
(38, 11.765188217163086)
(39, 12.452439308166504)
(40, 9.92991828918457)
(41, 7.642862319946289)
(4

In [62]:
topk_n, topk_i = model(x).data.topk(1)

In [80]:
count = 0
for i in xrange(Y_test.size):
    if topk_i[i][0] == Y_test[i]:
        count+=1

print(float(count)/float(Y_test.size))

0.291208791209


In [55]:
c = a.view(1092,1191,1)

In [8]:

m = nn.ConstantPad2d((1,0,1,0), 0)
input = m(input)
print(input)

Variable containing:
( 0  , 0  ,.,.) = 
  0.0000e+00  0.0000e+00  0.0000e+00  ...   0.0000e+00  0.0000e+00  0.0000e+00
  0.0000e+00  8.5739e+04  3.6875e+01  ...   0.0000e+00  0.0000e+00  1.0000e+00
  0.0000e+00  3.2376e+05  2.5887e+01  ...   0.0000e+00  0.0000e+00  1.0000e+00
                 ...                   ⋱                   ...                
  0.0000e+00  4.3537e+05  3.5166e+01  ...   0.0000e+00  0.0000e+00  1.0000e+00
  0.0000e+00  4.8975e+05  3.5734e+01  ...   0.0000e+00  0.0000e+00  1.0000e+00
  0.0000e+00  2.9089e+05  4.1883e+01  ...   0.0000e+00  1.0000e+00  0.0000e+00
[torch.FloatTensor of size 1x1x7276x1192]



In [60]:
c = Variable(c)
c

Variable containing:
( 0  ,.,.) = 
  3.6679e+05
  5.2216e+01
 -1.7245e+02
     ⋮      
  0.0000e+00
  1.0000e+00
  0.0000e+00

( 1  ,.,.) = 
  2.4482e+05
  4.8178e+01
 -1.2537e+02
     ⋮      
  0.0000e+00
  0.0000e+00
  1.0000e+00

( 2  ,.,.) = 
  1.1491e+05
  3.3355e+01
 -1.1902e+02
     ⋮      
  0.0000e+00
  1.0000e+00
  0.0000e+00
 ... 

(1089,.,.) = 
  2.0863e+05
  3.3971e+01
 -1.2113e+02
     ⋮      
  0.0000e+00
  0.0000e+00
  1.0000e+00

(1090,.,.) = 
  4.1969e+04
  3.3919e+01
 -1.1947e+02
     ⋮      
  0.0000e+00
  0.0000e+00
  1.0000e+00

(1091,.,.) = 
  1.1450e+05
  4.1919e+01
 -1.2485e+02
     ⋮      
  0.0000e+00
  1.0000e+00
  0.0000e+00
[torch.FloatTensor of size 1092x1191x1]

In [37]:
m = nn.Conv1d(16, 33, 3, stride=3)
input = Variable(torch.randn(1, 16, 50))
output = m(input)
output

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
  -0.1936 -0.6304 -0.8411 -1.2612 -0.0812  0.8946  0.0219 -0.1573 -0.6512
 -0.4543 -1.6682  0.2163 -0.7512 -1.5829  0.2780 -0.1479 -0.9394  0.0997
  1.4476 -0.1878 -0.1277  0.4280 -0.2548 -0.5982  0.0191 -0.6092 -0.2152
  0.9723  0.0906  0.0756  0.4512  0.4943  0.0362 -0.3273  0.1057 -0.0610
  1.1797  0.0102 -0.0353 -0.6266  0.9450  0.2118 -0.5839 -0.1127  0.4460
  0.7509 -0.0224  0.2855 -0.0995  0.5524 -0.3182  0.3746  0.4912  0.0287
  0.2101 -0.1660 -0.3724  0.6290  0.4723 -0.0508  0.5499  0.5503 -0.4621
 -0.9565  0.2953  0.1989  0.0417  0.6309  0.2004  1.1365  0.9270  0.0103
 -1.0803  0.8069  0.2605  0.0291  0.3395  0.1008 -0.6125  0.6509 -0.5378
  0.2490 -1.7430  0.0091  0.4654  0.2582 -0.4039 -0.5528  0.5561  1.1241
 -0.1867 -0.2212  0.9999  0.3781  0.1579  0.1973 -0.1380  0.4860  0.4885
 -0.4869 -0.5138 -0.6821  0.4750  0.3146  0.5517 -0.1379 -0.1481 -0.3867
 -0.2550  1.0557  0.3911 -0.0236  1.0585  0.7120  0.0993  0.5286 -0.3901


In [85]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
## load mnist dataset
use_cuda = torch.cuda.is_available()

root = './data'
download = False  # download MNIST dataset or not

trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=root, train=True, transform=trans, download=download)
test_set = dset.MNIST(root=root, train=False, transform=trans)

batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

print '==>>> total trainning batch number: {}'.format(len(train_loader))
print '==>>> total testing batch number: {}'.format(len(test_loader))


==>>> total trainning batch number: 600
==>>> total testing batch number: 100


In [133]:
class MLPNet(nn.Module):
    def __init__(self):
        super(MLPNet, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256,10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
all_losses = []
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
criteria = nn.CrossEntropyLoss()
model = MLPNet()
for epoch in range(10):
    ave_loss = 0
    for batch_idx, (x,y) in enumerate(train_loader):
        x, y = Variable(x), Variable(y)
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)
        # Compute and print loss
        loss = criterion(y_pred, y)
        ave_loss = ave_loss * 0.9 + loss.data[0] * 0.1
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (batch_idx+1) % 100 == 0 or (batch_idx+1) == len(train_loader):
            print '==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(
                epoch, batch_idx+1, ave_loss)

==>>> epoch: 0, batch index: 600, train loss: 2.309628
==>>> epoch: 1, batch index: 600, train loss: 2.306904
==>>> epoch: 2, batch index: 600, train loss: 2.309014
==>>> epoch: 3, batch index: 600, train loss: 2.311563
==>>> epoch: 4, batch index: 600, train loss: 2.306196
==>>> epoch: 5, batch index: 600, train loss: 2.309915
==>>> epoch: 6, batch index: 600, train loss: 2.308569
==>>> epoch: 7, batch index: 600, train loss: 2.312373
==>>> epoch: 8, batch index: 600, train loss: 2.309463
==>>> epoch: 9, batch index: 600, train loss: 2.309551


In [113]:
a = torch.range(1, 16)

/Users/ellise/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.3. Note that arange generates values in [start; end), not [start; end].
  """Entry point for launching an IPython kernel.


In [ ]:
for batch_idx, (x,y) in enumerate(test_loader):
        x, y = Variable(x), Variable(y)
        y_pred = model(x)
        